In [ ]:
import numpy as np
import sqlite3
import pandas as pd

# Função para conectar ao banco de dados SQLite
def conectar_banco_de_dados():
    return sqlite3.connect('projeto_teste.db')

# Carregar o dataset
df = pd.read_csv("diabetes.csv")

# Exibir informações do DataFrame
#print(df.shape)
#print(df.head())

# Criar conexão com o banco
conn = conectar_banco_de_dados()

# Salvar DataFrame no banco SQLite
df.to_sql("diabetes", conn, if_exists="replace", index=False)

query = "SELECT Age, Glucose, Outcome FROM diabetes WHERE Glucose > 190"
df_glicose_alta = pd.read_sql(query,conn)
print(df_glicose_alta)




# Fechar a conexão
conn.close()


In [ ]:
create_table_query = """
CREATE TABLE pacientes (
    Pregnancies INT,
    Glucose INT,
    BloodPressure INT,
    SkinThickness INT,
    Insulin INT,
    BMI DECIMAL(8, 2),
    DiabetesPedigreeFunction DECIMAL(8, 2),
    Age INT,
    Outcome INT
);
"""
conn = conectar_banco_de_dados()
cursor = conn.cursor()


cursor.execute(create_table_query)

conn.commit()
conn.close()

In [14]:
insert_query = """
INSERT INTO pacientes (Pregnancies, 
                       Glucose, 
                       BloodPressure, 
                       SkinThickness, 
                       Insulin, 
                       BMI, 
                       DiabetesPedigreeFunction, 
                       Age, 
                       Outcome) 
SELECT Pregnancies, 
       Glucose, 
       BloodPressure, 
       SkinThickness, 
       Insulin, 
       BMI, 
       DiabetesPedigreeFunction, 
       Age, 
       Outcome 
FROM diabetes WHERE Age > 50;
"""

conn = conectar_banco_de_dados()
cursor = conn.cursor()


cursor.execute(insert_query)

conn.commit()
conn.close()

In [ ]:
query = "SELECT * FROM pacientes"
conn = conectar_banco_de_dados()
consultar_pacientes = pd.read_sql(query,conn)
display(consultar_pacientes)

conn.close()

In [ ]:
%pip install -q ipython-sql